# Exercise 1: Agent Hello World - Solution

Complete implementation of a simple AI agent with tool calling.

## Setup

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from rich.console import Console

if not globals().get("ENV_LOADED", False):
    load_dotenv()
    ENV_LOADED = True
    env_status = "Loaded environment variables from .env"
else:
    env_status = "Environment variables already loaded"

console = Console()

console.print("\n[bold cyan]🤖 Exercise 1: Agent Hello World - Solution[/bold cyan]\n")
console.print(f"[green]✓[/green] {env_status}")

🤖 Exercise 1: Agent Hello World - Solution

✓ Loaded environment variables from .env

## Define Tool Function

In [2]:
def get_weather(location: str) -> str:
    """
    Tool function: Get current weather for a location.
    """
    weather_data = {
        "San Francisco": "sunny, 72°F",
        "New York": "cloudy, 65°F",
        "London": "rainy, 58°F",
        "Tokyo": "clear, 70°F",
    }
    weather = weather_data.get(location, "partly cloudy, 68°F")
    return f"The weather in {location} is {weather}"

## Define Tool Schema

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g., San Francisco",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

## Initialize OpenAI Client

In [4]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is not set. Add it to your environment or .env file.")

masked_key = f"{api_key[:4]}...{api_key[-4:]}" if len(api_key) >= 8 else "***masked***"
console.print(f"[green]✓[/green] OPENAI_API_KEY detected: {masked_key}")
client = OpenAI(api_key=api_key)
console.print("[green]✓[/green] OpenAI client initialized")

✓ OPENAI_API_KEY detected: sk-s...WyoA

✓ OpenAI client initialized

## Run the Agent

In [5]:
query = "What's the weather like in San Francisco?"
console.print(f"[yellow]Query:[/yellow] {query}\n")

# Initial API call
messages = [{"role": "user", "content": query}]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

# Check if model wants to call a tool
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    console.print("[green]🔧 Tool Called:[/green]")
    messages.append(response_message)
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = eval(tool_call.function.arguments)
        
        console.print(f"   {function_name}({function_args})")
        
        # Call our tool function
        if function_name == "get_weather":
            function_response = get_weather(**function_args)
            
            # Add tool response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    
    # Get final response
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    final_message = final_response.choices[0].message.content
else:
    final_message = response_message.content

console.print(f"\n[bold green]🤖 Agent Response:[/bold green]\n{final_message}\n")

Query: What's the weather like in San Francisco?

🔧 Tool Called:

get_weather({'location': 'San Francisco'})

🤖 Agent Response:
The weather in San Francisco is sunny, with a temperature of 72°F.